In [288]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import OneHotEncoder
import warnings
from sklearn.model_selection import train_test_split

In [289]:
data = pd.read_csv("traffic_data.csv")
features = data[['timestamp', 'intensity', 'velocity', 'lat', 'long', 'tweet count', 'weather','target']]
features['timestamp'] = pd.to_datetime(features['timestamp'])
features['month'] = features['timestamp'].dt.month
features['day'] = features['timestamp'].dt.day
features['hour'] = features['timestamp'].dt.hour
features = features.drop('timestamp', axis=1)
# target = data['target']
target[:10]

C:\Users\Anup\AppData\Local\Temp\ipykernel_26956\2603889052.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['timestamp'] = pd.to_datetime(features['timestamp'])
C:\Users\Anup\AppData\Local\Temp\ipykernel_26956\2603889052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['month'] = features['timestamp'].dt.month


0    1
1    1
2    1
3    1
4    1
5    1
6    0
7    0
8    0
9    0
Name: target, dtype: int64

In [290]:
features.head()

,intensity,velocity,lat,long,tweet count,weather,target,month,day,hour
0,5520,80,-3.674076,40.48064,9,partly-cloudy-night,1,5,11,9
1,5640,80,-3.674076,40.48064,32,partly-cloudy-night,1,5,11,9
2,5880,78,-3.674076,40.48064,51,partly-cloudy-night,1,5,11,9
3,5580,80,-3.674076,40.48064,32,partly-cloudy-night,1,5,11,9
4,4980,78,-3.674076,40.48064,55,partly-cloudy-night,1,5,11,10


In [291]:
unique_values_1 = features['weather'].unique()
print(unique_values_1)

['partly-cloudy-night' 'clear-night' 'clear-day' 'partly-cloudy-day'
 'cloudy' 'rain' 'fog']


In [292]:
weather_to_num = {"partly-cloudy-day":1,'partly-cloudy-night':2,"clear-night":3,"clear-day":4,"cloudy":5,"rain":6,'fog':7}


In [293]:
mapping = {value: i for i, value in enumerate(unique_values_1)}

In [294]:
features['weather'] = features['weather'].replace(mapping)

In [295]:
# data = features.groupby(['day','hour']).mean().reset_index()


In [296]:
features[:5]
lat = features['lat']
long = features['long']
features = features.drop(['lat','long'],axis = 1)
# features[:10]
print(features['day'].unique())
print(features['month'].unique())
print(features['hour'].unique())


[11 13 14 15 18 19 20 21 22 23 25 26]
[5]
[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23  0  1  2  3  4  5  6  7  8]


In [297]:
# target = features['target']
features = features.drop(['month'],axis=1)
# target

In [298]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
train_size = int(len(scaled_features) * 0.8)
train_data = scaled_features[:train_size]
target_train = scaled_features[:train_size]
target_test = scaled_features[train_size:]
test_data = scaled_features[train_size:]
np.shape(train_data)
np.shape(test_data)
# train_data,test_data,train_target,test_target = train_test_split(features,target,test_size=.2)
train_data
# test_data

array([[0.72222222, 0.54032258, 0.06666667, ..., 1.        , 0.        ,
        0.39130435],
       [0.73809524, 0.54032258, 0.25833333, ..., 1.        , 0.        ,
        0.39130435],
       [0.76984127, 0.52419355, 0.41666667, ..., 1.        , 0.        ,
        0.39130435],
       ...,
       [0.14285714, 0.51612903, 0.41666667, ..., 0.        , 0.        ,
        0.52173913],
       [0.08730159, 0.58064516, 0.425     , ..., 0.        , 0.        ,
        0.52173913],
       [0.08730159, 0.51612903, 0.41666667, ..., 0.        , 0.        ,
        0.52173913]])

In [260]:
# column_name = ['intensity', 'velocity', 'tweet count', 'weather','target','month','day','hour']

# df = pd.DataFrame(train_data,columns = column_name)
# target_column = df['target']
# target_column[:5]
# df2 = pd.DataFrame(test_data,columns = column_name)
# test_column = df2['target']
# print(np.shape(target_column),np.shape(test_column))
# col = ['intensity', 'velocity', 'tweet count', 'weather', 'target', 'month',
#        'day', 'hour']
# Min = [np.min(features[i]) for i in features.columns]
# Max = [np.max(features[i]) for i in features.columns]
# np.shape(Max)
# features.columns

In [299]:
# sequence_length = 20  # Choose an appropriate sequence length
# train_generator = TimeseriesGenerator(train_data, target_column, length=sequence_length, batch_size=32)
# test_generator = TimeseriesGenerator(test_data, test_column, length=sequence_length, batch_size=1)
# train_data = train_generator[0][0]
# target_data = train_generator[0][1]
# test_target_data = train_generator[0][1]
# test_data = test_generator[0][0]
# # np.shape(target_data)
sequence_length = 10  # Choose an appropriate sequence length
train_generator = TimeseriesGenerator(train_data, train_data, length=sequence_length, batch_size=32)
test_generator = TimeseriesGenerator(test_data, test_data, length=sequence_length, batch_size=1)
# np.shape(np.array(train_generator))

In [302]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(sequence_length, train_data.shape[1])))
model.add(Dense(train_data.shape[1],activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])



In [303]:
model.fit(train_generator, epochs=10)

# Generate predictions
predictions = model.predict(test_generator)

# Print the predictions
print(np.shape(predictions))

Epoch 1/10
151/151 [==============================] - 9s 7ms/step - loss: 0.0747 - accuracy: 0.2652
Epoch 2/10
151/151 [==============================] - 1s 6ms/step - loss: 0.0370 - accuracy: 0.6243
Epoch 3/10
151/151 [==============================] - 1s 6ms/step - loss: 0.0297 - accuracy: 0.6946
Epoch 4/10
151/151 [==============================] - 1s 6ms/step - loss: 0.0256 - accuracy: 0.7343
Epoch 5/10
151/151 [==============================] - 1s 6ms/step - loss: 0.0232 - accuracy: 0.7512
Epoch 6/10
151/151 [==============================] - 1s 6ms/step - loss: 0.0212 - accuracy: 0.7628
Epoch 7/10
151/151 [==============================] - 1s 6ms/step - loss: 0.0198 - accuracy: 0.7718
Epoch 8/10
151/151 [==============================] - 1s 7ms/step - loss: 0.0188 - accuracy: 0.7718
Epoch 9/10
151/151 [==============================] - 1s 7ms/step - loss: 0.0178 - accuracy: 0.7888
Epoch 10/10
1195/1195 [==============================] - 3s 2ms/step
(1195, 7)


In [265]:
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, train_data.shape[1]), return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(drop)
model.add(LSTM(units=64))
model.add(Dense(units=7, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, batch_size=32)

Epoch 1/10
151/151 [==============================] - 7s 17ms/step - loss: 6.3865 - accuracy: 0.2139
Epoch 2/10
151/151 [==============================] - 3s 20ms/step - loss: 7.0203 - accuracy: 0.2559
Epoch 3/10
151/151 [==============================] - 3s 21ms/step - loss: 7.0883 - accuracy: 0.2559
Epoch 4/10
151/151 [==============================] - 3s 17ms/step - loss: 7.1716 - accuracy: 0.2559
Epoch 5/10
151/151 [==============================] - 3s 18ms/step - loss: 7.5169 - accuracy: 0.2559
Epoch 6/10
151/151 [==============================] - 3s 17ms/step - loss: 7.8356 - accuracy: 0.2559
Epoch 7/10
151/151 [==============================] - 3s 17ms/step - loss: 8.1402 - accuracy: 0.2559
Epoch 8/10
151/151 [==============================] - 3s 17ms/step - loss: 8.2872 - accuracy: 0.2559
Epoch 9/10
151/151 [==============================] - 3s 17ms/step - loss: 8.6182 - accuracy: 0.2559
Epoch 10/10
151/151 [==============================] - 3s 17ms/step - loss: 8.9184 - accura

In [266]:
predictions = model.predict(test_generator)
predictions


1195/1195 [==============================] - 5s 3ms/step


array([[1.08677246e-01, 3.96519214e-01, 2.02298269e-01, ...,
        1.80073795e-04, 3.32652417e-04, 2.70852715e-01],
       [1.08678430e-01, 3.96520525e-01, 2.02298939e-01, ...,
        1.80074392e-04, 3.32653493e-04, 2.70849466e-01],
       [1.08678609e-01, 3.96521181e-01, 2.02297747e-01, ...,
        1.80073315e-04, 3.32654046e-04, 2.70849943e-01],
       ...,
       [1.08674869e-01, 3.96522611e-01, 2.02301562e-01, ...,
        1.80067087e-04, 3.32645111e-04, 2.70848840e-01],
       [1.08674869e-01, 3.96522611e-01, 2.02301562e-01, ...,
        1.80067087e-04, 3.32645111e-04, 2.70848840e-01],
       [1.08675599e-01, 3.96522254e-01, 2.02301383e-01, ...,
        1.80068309e-04, 3.32644791e-04, 2.70848602e-01]], dtype=float32)

In [264]:
# Fit the model
model.fit(train_generator, epochs=10)

# Generate predictions
predictions = model.predict(test_generator)

# Print the predictions
print(np.shape(predictions))

Epoch 1/10
151/151 [==============================] - 4s 4ms/step - loss: 0.0831 - accuracy: 0.4498
Epoch 2/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0245 - accuracy: 0.7223
Epoch 3/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0196 - accuracy: 0.7637
Epoch 4/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0175 - accuracy: 0.7816
Epoch 5/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0161 - accuracy: 0.7938
Epoch 6/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0153 - accuracy: 0.8055
Epoch 7/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0147 - accuracy: 0.8078
Epoch 8/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0143 - accuracy: 0.8107
Epoch 9/10
151/151 [==============================] - 1s 4ms/step - loss: 0.0140 - accuracy: 0.8107
Epoch 10/10
1195/1195 [==============================] - 2s 1ms/step
(1195, 7)


In [304]:
predictions = model.predict(train_generator)
predictions
    

151/151 [==============================] - 1s 3ms/step


array([[0.5609921 , 0.6292774 , 0.3340318 , ..., 0.08899622, 0.03981145,
        0.43484756],
       [0.56746835, 0.6303733 , 0.3489684 , ..., 0.07888908, 0.04410298,
        0.43227696],
       [0.5671727 , 0.6261454 , 0.35878068, ..., 0.08012041, 0.04583609,
        0.43345922],
       ...,
       [0.08203975, 0.6310552 , 0.39829096, ..., 0.055901  , 0.03250718,
        0.43036294],
       [0.08399286, 0.6286545 , 0.39085934, ..., 0.0563974 , 0.03338579,
        0.43128848],
       [0.08249477, 0.63001126, 0.38692775, ..., 0.05567548, 0.03453232,
        0.42724586]], dtype=float32)

In [274]:
predictions[0][0]* (7620 - 60) + 60
# check = train_data[0]
import pickle
# checka = [features[]]
# features['intensity']
with open('model.pkl','wb') as f:
    pickle.dump(model,f)

In [206]:
array = np.array(predictions)
# array[1]
finaldata = [(Min[i], Max[i])for i in range(len(array[0]))]
finaldata
Dict_to_map = {'intensity':(60,7620),'velocity':(13,37),'tweetcount':(1, 121),'weather':(0,6),'month':(0,1),'day':(5, 5),'hour':(11, 26)}

[(60, 7620), (13, 137), (1, 121), (0, 6), (0, 1), (5, 5), (11, 26)]

In [ ]:
# Assuming you have the true labels for the test data in 'true_labels' variable
true_labels = target

# Convert the predicted probabilities to binary predictions
binary_predictions = (target_predictions > 0.5).astype(int)

# Calculate accuracy for target predictions
target_accuracy = (binary_predictions == true_labels).mean()

# Generate predictions for the intermediate layer
intermediate_layer_output = intermediate_layer_model.predict(test_generator)

# Calculate accuracy for intermediate layer predictions
intermediate_accuracy = (intermediate_layer_output == true_labels).mean()

# Print the accuracies
print("Accuracy for target predictions:", target_accuracy)
print("Accuracy for intermediate layer predictions:", intermediate_accuracy)


In [ ]:
target_predictions[:10]

In [88]:
np.shape(additional_dense_predictions)

(1185, 7)

In [287]:

# Load and preprocess the data
# data = pd.read_csv('traffic_data.csv')  # Replace 'traffic_data.csv' with your dataset file
# target = data['congestion']  # Replace 'congestion' with your target variable column name
# features = data.drop(['congestion'], axis=1)  # Replace 'congestion' with your target variable column name

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)

# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(1, X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
print(np.shape(X_train))
# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# ll = np.reshape(np.array([5520,80,9,1,0,11,9]),(1,1,7))

(4817, 1, 7)
Epoch 1/50
151/151 [==============================] - 3s 5ms/step - loss: 0.4899 - accuracy: 0.7895
Epoch 2/50
151/151 [==============================] - 1s 6ms/step - loss: 0.0892 - accuracy: 1.0000
Epoch 3/50
151/151 [==============================] - 1s 5ms/step - loss: 0.0163 - accuracy: 1.0000
Epoch 4/50
151/151 [==============================] - 1s 6ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 5/50
151/151 [==============================] - 1s 5ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 6/50
151/151 [==============================] - 1s 4ms/step - loss: 0.0022 - accuracy: 1.0000
Epoch 7/50
151/151 [==============================] - 1s 5ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 8/50
151/151 [==============================] - 1s 4ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 9/50
151/151 [==============================] - 1s 5ms/step - loss: 7.8526e-04 - accuracy: 1.0000
Epoch 10/50
151/151 [==============================] - 1s 5ms/step - loss: 6.0565e-

In [280]:

ll = np.reshape(np.array([5520,80,9,1,0,11,9]),(1,1,7))
aa  = model.predict([ll])
aa

ValueError: in user code:

    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\Anup\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Anup

In [278]:
np.shape(X_train)

(4817, 1, 7)

In [245]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

# Load and preprocess the data
data = pd.read_csv('traffic_data.csv')
feature = data[['timestamp', 'intensity', 'velocity', 'lat', 'long', 'tweet count', 'weather','target']]
feature['timestamp'] = pd.to_datetime(feature['timestamp'])
feature['month'] = feature['timestamp'].dt.month
feature['day'] = feature['timestamp'].dt.day
feature['hour'] = feature['timestamp'].dt.hour
feature = feature.drop('timestamp', axis=1)# Replace 'traffic_data.csv' with your dataset file
target = feature['target']  # Replace 'congestion' with your target variable column name
features = feature.drop(['target'], axis=1)  # Replace 'congestion' with your target variable column name
feature['weather'] = feature['weather'].replace(mapping)
# Create a base classifier
base_model = RandomForestClassifier()

# Perform recursive feature elimination with cross-validation
rfecv = RFECV(estimator=base_model)
rfecv.fit(feature, target)

# Get the selected features
selected_features = feature.columns[rfecv.support_]

# Print selected features
print(selected_features)

C:\Users\Anup\AppData\Local\Temp\ipykernel_26956\646559822.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature['timestamp'] = pd.to_datetime(feature['timestamp'])
C:\Users\Anup\AppData\Local\Temp\ipykernel_26956\646559822.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature['month'] = feature['timestamp'].dt.month


Index(['target'], dtype='object')


In [246]:
correlation = features.corrwith(target)
print(correlation)

intensity      0.232245
velocity      -0.519906
lat           -0.046211
long          -0.032988
tweet count    0.134211
month               NaN
day            0.066006
hour           0.142255
dtype: float64


In [285]:
# with open('model.pkl')
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
print(X_test.shape[0], 1, X_test.shape[1])
# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
X_test.shape[1]

1205 1 7


1

In [305]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))


# the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))  # layer for regularization
model.add(LSTM(128))
model.add(Dropout(0.2))  # layer for regularization
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# validation mechanism
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train 
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

#testing
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/50
121/121 [==============================] - 11s 48ms/step - loss: 0.5094 - accuracy: 0.7355 - val_loss: 0.2004 - val_accuracy: 0.9180
Epoch 2/50
121/121 [==============================] - 4s 33ms/step - loss: 0.1809 - accuracy: 0.9245 - val_loss: 0.0823 - val_accuracy: 0.9606
Epoch 3/50
121/121 [==============================] - 4s 30ms/step - loss: 0.0167 - accuracy: 0.9925 - val_loss: 0.0056 - val_accuracy: 0.9990
Epoch 4/50
121/121 [==============================] - 4s 32ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 0.9990
Epoch 5/50
121/121 [==============================] - 4s 35ms/step - loss: 3.0974e-04 - accuracy: 1.0000 - val_loss: 9.2687e-04 - val_accuracy: 1.0000
Epoch 6/50
121/121 [==============================] - 4s 33ms/step - loss: 1.9132e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 0.9990
Epoch 7/50
121/121 [==============================] - 4s 32ms/step - loss: 0.0111 - accuracy: 0.9966 - val_loss: 0.0036 - val_acc

In [ ]:
# pip install schedule
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
import schedule
import time

# Define your data and model
# ...

# Define a function to train the model
def train_model():
    # Data preprocessing
    print("hihihihi")
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(features)
    X_train, X_test, y_train, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    X_val = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)

# Define the schedule to train the model every 5 minutes
schedule.every(5).minutes.do(train_model)

# Run the scheduler indefinitely
while True:
    schedule.run_pending()
    time.sleep(1)


Epoch 1/50
121/121 [==============================] - 9s 37ms/step - loss: 0.5082 - accuracy: 0.7423 - val_loss: 0.2310 - val_accuracy: 0.9180
Epoch 2/50
121/121 [==============================] - 3s 29ms/step - loss: 0.1546 - accuracy: 0.9343 - val_loss: 0.0351 - val_accuracy: 0.9896
Epoch 3/50
121/121 [==============================] - 4s 32ms/step - loss: 0.0147 - accuracy: 0.9961 - val_loss: 0.0070 - val_accuracy: 0.9979
Epoch 4/50
121/121 [==============================] - 3s 28ms/step - loss: 0.0116 - accuracy: 0.9958 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 5/50
121/121 [==============================] - 3s 28ms/step - loss: 9.8381e-04 - accuracy: 0.9997 - val_loss: 0.0018 - val_accuracy: 0.9990
Epoch 6/50
121/121 [==============================] - 3s 29ms/step - loss: 4.1217e-04 - accuracy: 1.0000 - val_loss: 8.4384e-04 - val_accuracy: 1.0000
Epoch 7/50
121/121 [==============================] - 4s 33ms/step - loss: 2.6856e-04 - accuracy: 1.0000 - val_loss: 7.5634e-04 - 

121/121 [==============================] - 5s 40ms/step - loss: 0.0113 - accuracy: 0.9966 - val_loss: 0.0175 - val_accuracy: 0.9927
Epoch 5/50
121/121 [==============================] - 5s 40ms/step - loss: 0.0029 - accuracy: 0.9990 - val_loss: 0.0019 - val_accuracy: 0.9990
Epoch 6/50
121/121 [==============================] - 5s 41ms/step - loss: 9.2459e-04 - accuracy: 1.0000 - val_loss: 8.1911e-04 - val_accuracy: 1.0000
Epoch 7/50
121/121 [==============================] - 5s 40ms/step - loss: 4.2743e-04 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 0.9990
Epoch 8/50
121/121 [==============================] - 5s 40ms/step - loss: 0.0066 - accuracy: 0.9974 - val_loss: 0.0015 - val_accuracy: 0.9990
Epoch 9/50
121/121 [==============================] - 5s 40ms/step - loss: 9.0898e-04 - accuracy: 0.9997 - val_loss: 0.0017 - val_accuracy: 0.9990
Epoch 10/50
121/121 [==============================] - 5s 40ms/step - loss: 3.0418e-04 - accuracy: 1.0000 - val_loss: 8.5309e-04 - val_ac

121/121 [==============================] - 9s 38ms/step - loss: 0.4478 - accuracy: 0.7734 - val_loss: 0.2545 - val_accuracy: 0.8890
Epoch 2/50
121/121 [==============================] - 3s 28ms/step - loss: 0.1417 - accuracy: 0.9450 - val_loss: 0.0266 - val_accuracy: 0.9907
Epoch 3/50
121/121 [==============================] - 4s 30ms/step - loss: 0.0073 - accuracy: 0.9984 - val_loss: 0.0082 - val_accuracy: 0.9990
Epoch 4/50
121/121 [==============================] - 3s 28ms/step - loss: 0.0154 - accuracy: 0.9943 - val_loss: 0.0040 - val_accuracy: 0.9990
Epoch 5/50
121/121 [==============================] - 3s 28ms/step - loss: 0.0028 - accuracy: 0.9992 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 6/50
121/121 [==============================] - 3s 27ms/step - loss: 5.5120e-04 - accuracy: 1.0000 - val_loss: 8.1066e-04 - val_accuracy: 1.0000
Epoch 7/50
121/121 [==============================] - 3s 27ms/step - loss: 4.3243e-04 - accuracy: 1.0000 - val_loss: 7.6633e-04 - val_accuracy: 1

121/121 [==============================] - 3s 29ms/step - loss: 0.0072 - accuracy: 0.9971 - val_loss: 0.0013 - val_accuracy: 0.9990
Epoch 5/50
121/121 [==============================] - 4s 29ms/step - loss: 3.4149e-04 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 0.9990
Epoch 6/50
121/121 [==============================] - 4s 29ms/step - loss: 1.3715e-04 - accuracy: 1.0000 - val_loss: 9.3536e-04 - val_accuracy: 0.9990
Epoch 7/50
121/121 [==============================] - 4s 29ms/step - loss: 1.4347e-04 - accuracy: 1.0000 - val_loss: 2.4671e-04 - val_accuracy: 1.0000
Epoch 8/50
121/121 [==============================] - 4s 31ms/step - loss: 7.1712e-05 - accuracy: 1.0000 - val_loss: 4.9689e-04 - val_accuracy: 1.0000
Epoch 9/50
121/121 [==============================] - 4s 30ms/step - loss: 5.8790e-05 - accuracy: 1.0000 - val_loss: 2.9779e-04 - val_accuracy: 1.0000
Epoch 10/50
121/121 [==============================] - 4s 29ms/step - loss: 4.5080e-05 - accuracy: 1.0000 - val_loss:

121/121 [==============================] - 4s 30ms/step - loss: 1.6393e-04 - accuracy: 1.0000 - val_loss: 8.3208e-04 - val_accuracy: 1.0000
Epoch 8/50
121/121 [==============================] - 4s 32ms/step - loss: 1.3820e-04 - accuracy: 1.0000 - val_loss: 6.9106e-04 - val_accuracy: 1.0000
Epoch 9/50
121/121 [==============================] - 4s 32ms/step - loss: 1.1304e-04 - accuracy: 1.0000 - val_loss: 6.6524e-04 - val_accuracy: 1.0000
Epoch 10/50
121/121 [==============================] - 4s 32ms/step - loss: 9.7808e-05 - accuracy: 1.0000 - val_loss: 3.8334e-04 - val_accuracy: 1.0000
Epoch 11/50
121/121 [==============================] - 4s 30ms/step - loss: 7.2489e-05 - accuracy: 1.0000 - val_loss: 3.8703e-04 - val_accuracy: 1.0000
Epoch 12/50
121/121 [==============================] - 2s 20ms/step - loss: 6.0599e-05 - accuracy: 1.0000 - val_loss: 2.3916e-04 - val_accuracy: 1.0000
Epoch 13/50
121/121 [==============================] - 2s 19ms/step - loss: 5.0783e-05 - accuracy: 1.0

...

Epoch 35/50
121/121 [==============================] - 7s 62ms/step - loss: 4.3933e-06 - accuracy: 1.0000 - val_loss: 3.2042e-05 - val_accuracy: 1.0000
Epoch 36/50
121/121 [==============================] - 7s 62ms/step - loss: 4.1936e-06 - accuracy: 1.0000 - val_loss: 2.8823e-05 - val_accuracy: 1.0000
Epoch 37/50
121/121 [==============================] - 7s 60ms/step - loss: 4.3800e-06 - accuracy: 1.0000 - val_loss: 2.7071e-05 - val_accuracy: 1.0000
Epoch 38/50
121/121 [==============================] - 7s 61ms/step - loss: 3.7760e-06 - accuracy: 1.0000 - val_loss: 2.8298e-05 - val_accuracy: 1.0000
Epoch 39/50
121/121 [==============================] - 7s 61ms/step - loss: 4.3620e-06 - accuracy: 1.0000 - val_loss: 3.1733e-05 - val_accuracy: 1.0000
Epoch 40/50
121/121 [==============================] - 7s 61ms/step - loss: 3.1609e-06 - accuracy: 1.0000 - val_loss: 2.4239e-05 - val_accuracy: 1.0000
Epoch 41/50
121/121 [==============================] - 7s 61ms/step - loss: 4.2204e-06 -

Epoch 11/50
121/121 [==============================] - 10s 80ms/step - loss: 7.7643e-05 - accuracy: 1.0000 - val_loss: 4.4976e-04 - val_accuracy: 1.0000
Epoch 12/50
121/121 [==============================] - 9s 78ms/step - loss: 9.0887e-05 - accuracy: 1.0000 - val_loss: 7.9628e-04 - val_accuracy: 1.0000
Epoch 13/50
121/121 [==============================] - 9s 75ms/step - loss: 6.1219e-05 - accuracy: 1.0000 - val_loss: 4.0904e-04 - val_accuracy: 1.0000
Epoch 14/50
121/121 [==============================] - 10s 79ms/step - loss: 6.2767e-05 - accuracy: 1.0000 - val_loss: 5.6987e-04 - val_accuracy: 1.0000
Epoch 15/50
121/121 [==============================] - 9s 76ms/step - loss: 4.6981e-05 - accuracy: 1.0000 - val_loss: 3.2483e-04 - val_accuracy: 1.0000
Epoch 16/50
121/121 [==============================] - 9s 77ms/step - loss: 3.5540e-05 - accuracy: 1.0000 - val_loss: 1.6098e-04 - val_accuracy: 1.0000
Epoch 17/50
121/121 [==============================] - 9s 72ms/step - loss: 3.1868e-05

121/121 [==============================] - 9s 70ms/step - loss: 3.9605e-05 - accuracy: 1.0000 - val_loss: 1.0918e-04 - val_accuracy: 1.0000
Epoch 15/50
121/121 [==============================] - 9s 76ms/step - loss: 3.5396e-05 - accuracy: 1.0000 - val_loss: 8.3814e-05 - val_accuracy: 1.0000
Epoch 16/50
121/121 [==============================] - 9s 72ms/step - loss: 2.7957e-05 - accuracy: 1.0000 - val_loss: 6.7087e-05 - val_accuracy: 1.0000
Epoch 17/50
121/121 [==============================] - 9s 73ms/step - loss: 2.3484e-05 - accuracy: 1.0000 - val_loss: 7.6059e-05 - val_accuracy: 1.0000
Epoch 18/50
121/121 [==============================] - 9s 73ms/step - loss: 2.4616e-05 - accuracy: 1.0000 - val_loss: 7.9212e-05 - val_accuracy: 1.0000
Epoch 19/50
121/121 [==============================] - 10s 83ms/step - loss: 1.9349e-05 - accuracy: 1.0000 - val_loss: 5.5046e-05 - val_accuracy: 1.0000
Epoch 20/50
121/121 [==============================] - 9s 72ms/step - loss: 1.8653e-05 - accuracy: 

121/121 [==============================] - 15s 127ms/step - loss: 1.2999e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 0.9990
Epoch 6/50
121/121 [==============================] - 15s 128ms/step - loss: 9.2316e-05 - accuracy: 1.0000 - val_loss: 0.0015 - val_accuracy: 0.9990
Epoch 7/50
121/121 [==============================] - 16s 128ms/step - loss: 7.8112e-05 - accuracy: 1.0000 - val_loss: 9.8879e-04 - val_accuracy: 0.9990
Epoch 8/50
121/121 [==============================] - 16s 133ms/step - loss: 5.7312e-05 - accuracy: 1.0000 - val_loss: 6.0760e-04 - val_accuracy: 1.0000
Epoch 9/50
121/121 [==============================] - 16s 136ms/step - loss: 4.4675e-05 - accuracy: 1.0000 - val_loss: 8.5329e-04 - val_accuracy: 0.9990
Epoch 10/50
121/121 [==============================] - 16s 129ms/step - loss: 3.1876e-05 - accuracy: 1.0000 - val_loss: 6.1212e-04 - val_accuracy: 1.0000
Epoch 11/50
121/121 [==============================] - 20s 162ms/step - loss: 2.8629e-05 - accuracy:

121/121 [==============================] - 14s 119ms/step - loss: 1.2526e-05 - accuracy: 1.0000 - val_loss: 1.3904e-04 - val_accuracy: 1.0000
Epoch 22/50
121/121 [==============================] - 14s 116ms/step - loss: 1.2277e-05 - accuracy: 1.0000 - val_loss: 1.8080e-04 - val_accuracy: 1.0000
Epoch 23/50
121/121 [==============================] - 14s 115ms/step - loss: 1.1222e-05 - accuracy: 1.0000 - val_loss: 1.8769e-04 - val_accuracy: 1.0000
Epoch 24/50
121/121 [==============================] - 14s 116ms/step - loss: 1.0043e-05 - accuracy: 1.0000 - val_loss: 1.5948e-04 - val_accuracy: 1.0000
Epoch 25/50
121/121 [==============================] - 14s 115ms/step - loss: 9.8639e-06 - accuracy: 1.0000 - val_loss: 1.5284e-04 - val_accuracy: 1.0000
Epoch 26/50
38/38 [==============================] - 1s 37ms/step - loss: 8.3420e-06 - accuracy: 1.0000
Test Loss: 8.34196362120565e-06
Test Accuracy: 1.0
hihihihi
Epoch 1/50
121/121 [==============================] - 30s 138ms/step - loss: 0